In [6]:
import pandas as pd
import numpy as np
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from btm.ml_logic.pipe_model import pipe
from btm.ml_logic.data_predictions import load_data_predictions
from btm.ml_logic.data import load_data
from datetime import datetime
import pickle
import os
from io import BytesIO
import plotly
import nbformat

In [ ]:
gdp = pickle.load(open('Final_GDP_Interp.pkl',"rb"))
spx = pickle.load(open('spx_final_pickle.pkl',"rb"))

In [7]:
    #turning our prediction.csv into a DataFrame

    model = pickle.load(open('spx_final_pickle.pkl',"rb"))
    #X_dropped, y_dropped = load_data_predictions('predict_set_w_btm.csv','SPX Index ')
    X_dropped, y_dropped = load_data_predictions('total_df_w_btm.csv','SPX Index ')

    y_pred = model.predict(X_dropped)
    dict_out = {}
    for i, y in enumerate(y_pred):
        dict_out[X_dropped.index[i].to_pydatetime().strftime('%Y-%m-%d')] = \
            float(y)

    predictions_life = model.predict(X_dropped)

    predictions_life = pd.Series(predictions_life)
    predictions_life.index = y_dropped.index

    df_plot_life = pd.DataFrame()
    btm_gdp  = X_dropped['Final_GDP_Interp']

    df_plot_life = pd.concat([y_dropped, predictions_life, btm_gdp], axis=1)
    df_plot_life.info()

    df_plot_life['Market'] = df_plot_life['SPX Index ']
    df_plot_life['BTM Model'] = df_plot_life[0]
    df_plot_life['Diff'] = df_plot_life['SPX Index '] - df_plot_life[0]
    df_plot_life = df_plot_life.drop(columns=['SPX Index ', 0])
    df_plot_life['Diff Z-score'] = (df_plot_life['Diff'] - df_plot_life['Diff'].mean()) / df_plot_life['Diff'].std()

    conditions = [
    (df_plot_life['Diff Z-score'] > 2.5),
    (df_plot_life['Diff Z-score'] > 1.5),
    (df_plot_life['Diff Z-score'] < -2.5),
    (df_plot_life['Diff Z-score'] < -1.5)
    ]

    df_plot_life['BTM GDP'] = df_plot_life['Final_GDP_Interp'].fillna(method='ffill')
    df_plot_life = df_plot_life.drop(df_plot_life.columns[0], axis=1)

    values = ['Strong Sell', 'Sell', 'Strong Buy','Buy']

    # Create a new column 'Action' based on conditions and values
    df_plot_life['Action'] = np.select(conditions, values, default='Fair Value')
    print(df_plot_life)
    # Print the updated DataFrame
    print(df_plot_life)


    all_predictions = {
        'title' :'SPX Index Predictions',
        'df' : df_plot_life.to_dict()
    }


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2584 entries, 2014-05-01 to 2024-03-25
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SPX Index         2584 non-null   float64
 1   0                 2584 non-null   float32
 2   Final_GDP_Interp  2582 non-null   float64
dtypes: float32(1), float64(2)
memory usage: 70.7 KB
             Market    BTM Model        Diff  Diff Z-score   BTM GDP  \
Dates                                                                  
2014-05-01  1883.68  1902.226196  -18.546196     -0.172858  0.110000   
2014-05-02  1881.14  1895.911255  -14.771255     -0.110979  0.152667   
2014-05-05  1884.66  1900.315674  -15.655674     -0.125476  0.280667   
2014-05-06  1867.72  1891.458252  -23.738252     -0.257966  0.323333   
2014-05-07  1878.21  1896.537354  -18.327354     -0.169270  0.366000   
...             ...          ...         ...           ...       ...   
2024-03-19  

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Creating a figure with secondary_y axis (two y-axes)
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Function to generate hover text including the 'Action'
def create_hover_text(df):
    return [f"{date.strftime('%Y-%m-%d')}<br>Market: {market}<br>BTM Model: {model}<br>Action: {action}"
            for date, market, model, action in zip(df.index, df['Market'], df['BTM Model'], df['Action'])]

# Custom hover text for all data points
hover_text = create_hover_text(df_plot_life)

# Adding Market and BTM Model as line plots with custom hover info
fig.add_trace(
    go.Scatter(x=df_plot_life.index, y=df_plot_life['Market'], name='Market', mode='lines',
               line=dict(color='blue'), text=hover_text, hoverinfo='text'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_plot_life.index, y=df_plot_life['BTM Model'], name='BTM Model', mode='lines',
               line=dict(color='green'), text=hover_text, hoverinfo='text'),
    secondary_y=False,
)

# Modifying the 'Diff Z-score' bar chart to include 'Action' in hover
fig.add_trace(
    go.Bar(x=df_plot_life.index, y=df_plot_life['Diff Z-score'], name='Signal Strength',
           marker_color='pink', opacity=0.5, width=0.5,
           marker_line_color='pink', marker_line_width=1,
           text=hover_text, hoverinfo='text'),  # Include custom hover info
    secondary_y=True,
)

# Adding titles and labels
fig.update_layout(
    title_text="S&P500 vs BTM Model with Signal Strength",
    plot_bgcolor='white', paper_bgcolor='white',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Value'),
    yaxis2=dict(title='Signal Strength')
)

# Show plot
fig.show()
